# Sugarcane Disease Classification Training

This notebook trains a YOLOv11n-cls model for sugarcane disease classification.

**Model**: YOLOv11n-cls (nano classification - optimized for mobile/edge)

**Classes**: Healthy, Mosaic, Redrot, Rust, Yellow

**Hardware**: Run this on Google Colab with GPU enabled (Runtime → Change runtime type → GPU)

## Step 1: Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required packages
!pip install -q ultralytics kagglehub

# Import libraries
from ultralytics import YOLO
import kagglehub
import os
from pathlib import Path
import shutil
from IPython.display import Image, display

print("✅ Setup complete!")

## Step 2: Download Dataset

⚠️ **Important**: You need Kaggle API credentials:
1. Go to https://www.kaggle.com/settings
2. Click "Create New API Token"
3. Upload the `kaggle.json` file to Colab (Files panel on left)
4. Run the cell below

In [ ]:
# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle credentials configured!")

In [ ]:
# Download the dataset
print("📥 Downloading sugarcane disease dataset...")
dataset_path = kagglehub.dataset_download("nirmalsankalana/sugarcane-leaf-disease-dataset")
print(f"✅ Dataset downloaded to: {dataset_path}")

## Step 3: Prepare Dataset

Organize images into YOLO classification format:
```
dataset/
├── train/
│   ├── Healthy/
│   ├── Mosaic/
│   ├── Redrot/
│   ├── Rust/
│   └── Yellow/
├── val/
└── test/
```

In [ ]:
import random
from collections import defaultdict

# Configuration
CLASS_NAMES = ['Healthy', 'Mosaic', 'Redrot', 'Rust', 'Yellow']
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1
RANDOM_SEED = 42

random.seed(RANDOM_SEED)

# Discover images in the downloaded dataset
print("🔍 Discovering images...")
class_images = defaultdict(list)
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if Path(file).suffix.lower() in image_extensions:
            file_path = Path(root) / file
            folder_name = Path(root).name.lower()
            
            # Detect class from folder or filename
            for class_name in CLASS_NAMES:
                if class_name.lower() in folder_name or class_name.lower() in file.lower():
                    class_images[class_name].append(file_path)
                    break

# Print statistics
print("\n📊 Dataset Statistics:")
for class_name in CLASS_NAMES:
    count = len(class_images[class_name])
    print(f"   {class_name}: {count} images")
print(f"   Total: {sum(len(imgs) for imgs in class_images.values())} images")

In [ ]:
# Create directory structure and split dataset
output_dir = Path('/content/sugarcane_dataset')

print("\n✂️  Splitting and organizing dataset...")

for split in ['train', 'val', 'test']:
    for class_name in CLASS_NAMES:
        (output_dir / split / class_name).mkdir(parents=True, exist_ok=True)

# Split and copy images
stats = defaultdict(lambda: defaultdict(int))

for class_name, images in class_images.items():
    # Shuffle images
    shuffled = images.copy()
    random.shuffle(shuffled)
    
    # Calculate split points
    total = len(shuffled)
    train_end = int(total * TRAIN_RATIO)
    val_end = train_end + int(total * VAL_RATIO)
    
    # Split
    splits = {
        'train': shuffled[:train_end],
        'val': shuffled[train_end:val_end],
        'test': shuffled[val_end:]
    }
    
    # Copy images
    for split_name, split_images in splits.items():
        for img_path in split_images:
            dest = output_dir / split_name / class_name / img_path.name
            shutil.copy2(img_path, dest)
            stats[split_name][class_name] += 1

# Print split statistics
print("\n📊 Split Statistics:")
for split in ['train', 'val', 'test']:
    print(f"\n{split.upper()}:")
    for class_name in CLASS_NAMES:
        print(f"   {class_name}: {stats[split][class_name]} images")
    print(f"   Total: {sum(stats[split].values())} images")

print(f"\n✅ Dataset organized at: {output_dir}")

## Step 4: Visualize Sample Images

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image as PILImage

# Display sample images from each class
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for idx, class_name in enumerate(CLASS_NAMES):
    class_dir = output_dir / 'train' / class_name
    sample_img = list(class_dir.glob('*'))[0]
    
    img = PILImage.open(sample_img)
    axes[idx].imshow(img)
    axes[idx].set_title(class_name, fontsize=14, fontweight='bold')
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

print("Sample images from each disease class")

## Step 5: Train YOLOv11n-cls Model

**Training Parameters**:
- Model: YOLOv11n-cls (nano - optimized for mobile)
- Image size: 224x224
- Batch size: 64
- Epochs: 100 (with early stopping)
- Augmentation: Enabled (automatic)

In [ ]:
# Initialize model
print("🤖 Loading YOLOv11n-cls pretrained model...")
model = YOLO('yolov11n-cls.pt')

print("✅ Model loaded successfully!")

In [ ]:
# Train the model
print("🚀 Starting training...\n")

results = model.train(
    data=str(output_dir),
    epochs=100,
    imgsz=224,
    batch=64,
    patience=15,       # Early stopping after 15 epochs without improvement
    save=True,
    device=0,          # Use GPU 0
    workers=8,
    optimizer='Adam',
    lr0=0.001,
    project='sugarcane_disease',
    name='yolov11n_cls',
    exist_ok=True,
    pretrained=True,
    verbose=True
)

print("\n✅ Training complete!")

## Step 6: View Training Results

In [ ]:
# Display training curves
from IPython.display import Image, display

results_dir = Path('sugarcane_disease/yolov11n_cls')

print("📈 Training Results:\n")

# Display results image
results_img = results_dir / 'results.png'
if results_img.exists():
    display(Image(filename=str(results_img)))
else:
    print("Results image not found")

In [ ]:
# Display confusion matrix
confusion_matrix_img = results_dir / 'confusion_matrix_normalized.png'
if confusion_matrix_img.exists():
    print("\n📊 Confusion Matrix:\n")
    display(Image(filename=str(confusion_matrix_img)))
else:
    print("Confusion matrix not found")

## Step 7: Validate Model

In [ ]:
# Load best model and validate
best_model = YOLO(results_dir / 'weights' / 'best.pt')

print("🔍 Validating model on test set...\n")
metrics = best_model.val(data=str(output_dir), split='test')

print(f"\n📊 Test Set Performance:")
print(f"   Accuracy (top1): {metrics.top1:.4f}")
print(f"   Accuracy (top5): {metrics.top5:.4f}")
print("\n✅ Validation complete!")

## Step 8: Test Predictions

In [ ]:
# Test on sample images from test set
test_images = []
for class_name in CLASS_NAMES:
    class_dir = output_dir / 'test' / class_name
    test_images.append(list(class_dir.glob('*'))[0])

# Run predictions
print("🔮 Running predictions on test samples...\n")

for img_path in test_images:
    results = best_model(img_path, verbose=False)
    
    # Get prediction
    probs = results[0].probs
    predicted_class = CLASS_NAMES[probs.top1]
    confidence = probs.top1conf.item()
    true_class = img_path.parent.name
    
    # Display result
    status = "✅" if predicted_class == true_class else "❌"
    print(f"{status} True: {true_class:10} | Predicted: {predicted_class:10} | Confidence: {confidence:.3f}")

print("\n✅ Predictions complete!")

## Step 9: Export Model for Deployment

Export to TFLite format for mobile/edge deployment

In [ ]:
# Export to TFLite (for mobile deployment)
print("📦 Exporting model to TFLite format...\n")

tflite_model = best_model.export(
    format='tflite',
    imgsz=224,
    int8=False,  # Set to True for INT8 quantization (smaller, faster)
)

print(f"\n✅ Model exported: {tflite_model}")

In [ ]:
# Export to ONNX (alternative format for various platforms)
print("📦 Exporting model to ONNX format...\n")

onnx_model = best_model.export(
    format='onnx',
    imgsz=224,
    simplify=True
)

print(f"\n✅ Model exported: {onnx_model}")

## Step 10: Download Models

Download the trained models to your local machine

In [ ]:
# Zip the results folder for download
!zip -r sugarcane_models.zip sugarcane_disease/yolov11n_cls/weights/

print("\n✅ Models packaged!")
print("\n📥 Download 'sugarcane_models.zip' from the Files panel (left sidebar)")
print("\nContents:")
print("   - best.pt (PyTorch model)")
print("   - last.pt (last checkpoint)")
print("   - best_saved_model/ (TFLite model)")
print("   - best.onnx (ONNX model)")

## Summary

✅ **Completed Steps**:
1. Downloaded sugarcane disease dataset (2,569 images)
2. Organized into train/val/test splits
3. Trained YOLOv11n-cls model
4. Validated on test set
5. Exported to TFLite and ONNX formats

🚀 **Next Steps**:
1. Download the trained models
2. Integrate into your mobile application
3. Test on real sugarcane leaves in the field
4. Collect more data to improve accuracy

📱 **Mobile Integration**:
- Use the TFLite model for Android/iOS
- Expected inference speed: 30-60 FPS
- Model size: ~5 MB

📖 **Resources**:
- [Ultralytics YOLO Docs](https://docs.ultralytics.com)
- [TFLite Integration Guide](https://www.tensorflow.org/lite)
- [YOLO Classification Guide](https://docs.ultralytics.com/tasks/classify/)